# Preprocesamiento de Video con S3

Este notebook procesa un video almacenado en S3, extrae frames y los guarda como imágenes individuales en otro bucket de S3.

## 1. Configuración Inicial
Importación de bibliotecas necesarias y configuración del cliente AWS S3.

In [ ]:
# Instalar dependencias en terminal
pip install opencv-python-headless
pip install boto3
pip install tqdm

In [ ]:
# Importar las bibliotecas necesarias
import boto3      # SDK de AWS para Python
import cv2        # OpenCV para procesamiento de video
import json       # Manejo de JSON
import time       # Control de tiempo
from tqdm import tqdm  # Barra de progreso
import os        # Operaciones de sistema de archivos

# Configuración general
CONFIG = {
    # Configuración de AWS S3
    "AWS_REGION": "<REGION>",              # Región de AWS
    "INPUT_BUCKET": "<INPUT_BUCKET>",         # Bucket con el video
    "OUTPUT_BUCKET": "<OUTPUT_BUCKET>",       # Bucket para frames
    "VIDEO_KEY": "video.mp4",              # Nombre del video en S3
    "OUTPUT_FOLDER": "frames/",             # Carpeta destino para frames
    
    # Configuración de procesamiento
    "TEMP_DIR": "/tmp",                     # Directorio temporal
    "FRAME_INTERVAL": 1,                    # Intervalo entre frames (segundos)
    "FRAME_FORMAT": "jpg",                  # Formato de imagen
    "FRAME_QUALITY": 95,                    # Calidad de imagen (1-100)
}

# Inicializar cliente S3
try:
    s3 = boto3.client('s3', region_name=CONFIG["AWS_REGION"])
except Exception as e:
    print(f"Error al inicializar cliente S3: {str(e)}")
    raise

## 2. Función de Carga a S3
Definición de la función auxiliar para cargar archivos a S3. Esta función se utilizará para subir cada frame procesado.

In [ ]:
def upload_file(file_name: str, bucket: str, object_name: str=None):
    """
    Sube un archivo a un bucket de Amazon S3.
    
    Args:
        file_name (str): Ruta del archivo local
        bucket (str): Nombre del bucket S3
        object_name (str): Nombre del objeto en S3 (opcional)
    """
    if object_name is None:
        object_name = os.path.basename(file_name)
    try:
        s3.upload_file(file_name, bucket, object_name)
        return True
    except Exception as e:
        print(f"Error al subir archivo: {str(e)}")
        return False

## 3. Clase de Procesamiento de Video
Clase para procesar videos desde S3 y extraer frames

In [ ]:
def process_video():
    """
    Procesa el video y extrae frames
    """
    try:
        # Preparar rutas
        local_video_path = os.path.join(CONFIG["TEMP_DIR"], CONFIG["VIDEO_KEY"])
        
        # Descargar video
        print("Descargando video...")
        s3.download_file(
            CONFIG["INPUT_BUCKET"],
            CONFIG["VIDEO_KEY"],
            local_video_path
        )
        
        # Abrir video
        cap = cv2.VideoCapture(local_video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_interval = fps * CONFIG["FRAME_INTERVAL"]
        
        print(f"FPS: {fps}")
        print(f"Total frames: {total_frames}")
        
        # Procesar frames
        with tqdm(total=total_frames//frame_interval, desc="Procesando frames") as pbar:
            frame_count = 0
            while cap.isOpened():
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)
                ret, frame = cap.read()
                
                if not ret:
                    break
                
                # Calcular tiempo del frame
                current_second = frame_count * CONFIG["FRAME_INTERVAL"]
                current_minute = current_second // 60
                current_second %= 60
                
                # Crear nombre del frame
                frame_name = f"frame_{frame_count:04d}_min{current_minute:02d}_sec{current_second:02d}.{CONFIG['FRAME_FORMAT']}"
                local_frame_path = os.path.join(CONFIG["TEMP_DIR"], frame_name)
                
                # Guardar frame
                cv2.imwrite(
                    local_frame_path, 
                    frame, 
                    [cv2.IMWRITE_JPEG_QUALITY, CONFIG["FRAME_QUALITY"]]
                )
                
                # Subir a S3
                s3_path = os.path.join(CONFIG["OUTPUT_FOLDER"], frame_name)
                upload_file(local_frame_path, CONFIG["OUTPUT_BUCKET"], s3_path)
                
                # Limpiar
                os.remove(local_frame_path)
                frame_count += 1
                pbar.update(1)
        
        # Liberar recursos
        cap.release()
        os.remove(local_video_path)
        return True
        
    except Exception as e:
        print(f"Error en el procesamiento: {str(e)}")
        return False

## 4. Función Principal

In [ ]:
def main():
    """
    Función principal
    """
    start_time = time.time()
    
    success = process_video()
    
    # Calcular tiempo total
    total_time = time.time() - start_time
    
    # Mostrar resultados
    result = {
        'statusCode': 200 if success else 500,
        'body': 'Procesamiento completado' if success else 'Error en procesamiento',
        'total_time_seconds': total_time
    }
    
    print(json.dumps(result, indent=2))

if __name__ == "__main__":
    main()